In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
orders = data['orders'].copy()

In [32]:
# Import olist data
from olist.load_data import load_all_data
data = load_all_data("data/")

In [33]:
from olist.preprocess import transformar_columnas_datetime
orders = transformar_columnas_datetime(orders)

In [34]:
#pasarlo a modulo
from olist.preprocess import tiempo_de_espera
orders = tiempo_de_espera(orders, is_delivered = True)
orders.head()

/Users/usuario/PycharmProjects/pythonProject5/UCEMA/07-entrenamiento/olist/preprocess.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['tiempo_de_espera'] = (orders_df['order_delivered_customer_date'] - orders_df[


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,tiempo_de_espera
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.436574
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.782037
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.394213
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13.208750
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2.873877


In [35]:
# crear real vs esperado
# ver que columnas hay
# igual que en NPS si algun numero en real_vs_esperado es negativo reemplazarlo por 0
from olist.preprocess import real_vs_esperado
orders = real_vs_esperado(orders, is_delivered = True)
orders['real_vs_esperado']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99436    0.0
99437    0.0
99438    0.0
99439    0.0
99440    0.0
Name: real_vs_esperado, Length: 96478, dtype: float64

In [36]:
orders = orders.rename(
    columns={"order_estimated_delivery_date": 'tiempo_de_espera_previsto', 'order_status': "status_de_la_orden"})
orders_2 = orders[
    ['order_id', 'tiempo_de_espera', 'tiempo_de_espera_previsto', 'real_vs_esperado', 'status_de_la_orden']]
orders_2

,order_id,tiempo_de_espera,tiempo_de_espera_previsto,real_vs_esperado,status_de_la_orden
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,2017-10-18,0.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,2018-08-13,0.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,2018-09-04,0.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,2017-12-15,0.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,2018-02-26,0.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.218009,2017-03-28,0.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.193727,2018-03-02,0.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.859421,2017-09-27,0.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17.086424,2018-02-15,0.0,delivered


### b) `puntaje_de_compra`
     retorna un dataframe con
        order_id, es_cinco_estrellas, es_una_estrella, review_score

recordar del examen `DataFrame.apply()` o tmb se puede usar`Series.map()`

In [37]:
reviews = data['order_reviews'].copy()
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [24]:
from olist.preprocess import puntaje_de_compra
puntaje_de_compra(reviews)

,order_id,es_cinco_estrellas,es_una_estrella,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


c) numero_de_productos:
 retorna un dataframe con
    order_id, numero_de_productos (numero_de_productos_por_orden)

In [52]:
items = data['order_items'].copy()
items.order_id.nunique()

98666

In [ ]:
from olist.preprocess import calcular_numero_productos
#fijarse que no uso solo un dataframe
productos = calcular_numero_productos(data)
productos